# SelectIndexBox

- The `SelectIndexBox` is a register box for doing Select operators in LCU using the `IndexBox` abstraction. It is therefore agnostic to the index method used.
- It takes `QubitPauliOperator` as an operator and breaks it into a sum of Pauli strings. 
- A `RegisterBox` is formed for each pauli string of pytket `Op`s.
- The phase of the coefficient of the pauli term is contained in the pytket `Op` as a `Unitary1Q` gate. This is because the prepare can only have positive magnitudes. Hence if this box is used in an LCU the Prepare must use the magnitide of the coefficient in polar cords and the phase is absorbed into each term in the select.
- The absorption of the phase and the generation of the register boxes is done in the `SerialLCUOperator` class.


In [24]:
from qtnmtts.circuits.select import SelectIndexBox
from qtnmtts.circuits.index.method import IndexDefault, IndexUnaryIteration
from qtnmtts.operators import ising_model
from pytket.circuit.display import render_circuit_jupyter
from pytket.passes import DecomposeMultiQubitsCX, RemoveRedundancies

In [25]:
n_state_qubits = 30
hamiltonian = ising_model(n_state_qubits, h=1, j=1)
index_method = IndexDefault()
select_box = SelectIndexBox(index_method, hamiltonian, n_state_qubits)

In [26]:
render_circuit_jupyter(select_box.get_circuit())

In [27]:
index_method = IndexUnaryIteration()
select_box = SelectIndexBox(index_method, hamiltonian, n_state_qubits)

In [28]:
from pytket.passes import DecomposeBoxes
circ = select_box.get_circuit()
DecomposeBoxes().apply(circ)
DecomposeMultiQubitsCX().apply(circ)
render_circuit_jupyter(circ)
circ.n_gates

2397

# Multiplexor Comparison

In [29]:
from qtnmtts.circuits.select import SelectMultiplexorBox

select_box = SelectMultiplexorBox(hamiltonian, n_state_qubits)
circ = select_box.get_circuit()
DecomposeBoxes().apply(circ)
RemoveRedundancies().apply(circ)
render_circuit_jupyter(circ)
circ.n_gates

6884

# Benchmarking Comparison

- multiplexor
- select index box (unary iteration)

In [30]:
from pytket.passes import DecomposeMultiQubitsCX, RemoveRedundancies

def circ_counts(box, str):

    circ = box.get_circuit()
    DecomposeBoxes().apply(circ)
    DecomposeMultiQubitsCX().apply(circ)
    RemoveRedundancies().apply(circ)

    two_q_gates =circ.n_2qb_gates()
    n_gates = circ.n_gates
    depth = circ.depth()

    return n_gates,two_q_gates ,depth, n_state_qubits, str

def gate_counts_multi(n_state_qubits):
    hamiltonian = ising_model(n_state_qubits, h=1, j=1)
    select_box = SelectMultiplexorBox(hamiltonian, n_state_qubits)
    return circ_counts(select_box,'multi')

def gate_counts_index(n_state_qubits):
    hamiltonian = ising_model(n_state_qubits, h=1, j=1)
    index_method = IndexUnaryIteration()
    select_box = SelectIndexBox(index_method, hamiltonian, n_state_qubits)
    return circ_counts(select_box,'index')
    


In [35]:
import pandas as pd
data = []
for n_state_qubits in range(2,52,4):
    data.append(gate_counts_multi(n_state_qubits))

for n_state_qubits in range(2,52,4):
    data.append(gate_counts_index(n_state_qubits))

df = pd.DataFrame(data, columns=['n_gates', 'two_q_gates', 'depth', 'n_state_qubits', 'method'])

In [36]:
df

,n_gates,two_q_gates,depth,n_state_qubits,method
0,32,16,26,2,multi
1,381,200,351,6,multi
2,1225,660,1174,10,multi
3,1657,912,1585,14,multi
4,4207,2348,4113,18,multi
5,5064,2856,4950,22,multi
6,5991,3364,5857,26,multi
7,6884,3872,6732,30,multi
8,15936,8796,15768,34,multi
9,17722,9816,17530,38,multi
